In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from time import time
import sys as sys
#folder = '/home/gosztolai/Dropbox/github/geometric_clustering'
folder = '/Users/adamgosztolai/Dropbox/github/geometric_clustering'
sys.path.append('/anaconda3/lib/python3.7/site-packages')
sys.path.append(folder + '/utils')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn import manifold, datasets
from sklearn.preprocessing import normalize
from scipy.spatial.distance import squareform
import os as os
import networkx as nx
from geometric_clustering import Geometric_Clustering
from graph_generator import generate_graph
from misc import load_curvature, save_curvature
from sklearn.manifold import MDS
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.decomposition import KernelPCA
from sklearn.utils.graph import graph_shortest_path
import yaml

In [10]:
graph_tpe = 'S'
params = yaml.load(open(folder + '/utils/graph_params.yaml','rb'), Loader=yaml.FullLoader)[graph_tpe]
os.chdir(folder + '/scripts/' +graph_tpe)
G = nx.read_gpickle(graph_tpe + ".gpickle")

Ggen = generate_graph(tpe=graph_tpe, params=params)

# initialise the code with parameters and graph 
gc = Geometric_Clustering(G)
load_curvature(gc)
gc.compute_node_curvature()

X = nx.get_node_attributes(G,'block')[0]
color = nx.get_node_attributes(G,'color')[0]
for i in range(1,gc.n):
    X = np.vstack([X,nx.get_node_attributes(G,'block')[i]])
    color = np.vstack([color,nx.get_node_attributes(G,'color')[i]])

#plots    
#color = np.squeeze(color)    
fig = plt.figure(figsize=(15, 8))
gs = fig.add_gridspec(gc.n_t//3+1, 3)

for i in range(gc.n_t):
    ax = fig.add_subplot(gs[i//3, np.mod(i,3)], projection='3d')
    #ax = fig.add_subplot(3i)
    color = gc.Kappa[:,i]
    #ax.scatter(X[:, 0], X[:, 1], X[:, 2], c='k', cmap=plt.cm.Spectral)
    
    for i,j in enumerate(gc.G.edges()):
 
        x = np.array((X[j[0]][0], X[j[1]][0]))
        y = np.array((X[j[0]][1], X[j[1]][1]))
        z = np.array((X[j[0]][2], X[j[1]][2]))
        
        # Plot the connecting lines
        ax.plot(x, y, z, c=color, alpha=0.5)
        
    ax.axis('tight')
    ax.view_init(4, -72)    

plt.show()

KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1134f5268> (for post_execute):


ValueError: RGBA sequence should have length 3 or 4

ValueError: RGBA sequence should have length 3 or 4

<Figure size 1080x576 with 2 Axes>

In [ ]:
#MDS alone    
A = nx.adjacency_matrix(Ggen).todense()
A[A==0]=10000
mds = manifold.MDS(2,dissimilarity = "precomputed")
Y = mds.fit_transform(A)    

nn=10
nbrs_ = NearestNeighbors(nn)
nbrs_.fit(X)
#training_data_ = nbrs_._fit_X

kernel_pca_ = KernelPCA(n_components=2,kernel="precomputed")
kng = kneighbors_graph(nbrs_, nn, mode='distance')

dist_matrix_ = graph_shortest_path(kng, directed=False)
dist_matrix_ = dist_matrix_ ** 2
dist_matrix_ *= -0.5

Y=kernel_pca_.fit_transform(dist_matrix_)
 
#curvature + MDS    
t = 0
Kappa = 0*np.ones([gc.n,gc.n])
np.fill_diagonal(Kappa, 0)
for i, e in enumerate(gc.G.edges):
    Kappa[e] = -np.exp(-gc.Kappa[:,t][i])

#se = manifold.SpectralEmbedding(2,n_neighbors=5)
#Ycurv = se.fit_transform(X)   
#Ycurv = mds.fit_transform(Kappa)
    
Kappa  = normalize(Kappa, norm = 'l1')    

#for i in range(gc.n):
#    for k in range(i,gc.n):
#        Kappa[k,i] = Kappa[i,k]
 
#Kappa = nx.adjacency_matrix(G, weight='curvature').todense()

#Laplacian
#D = np.diag(np.sum(np.abs(Kappa),0))
#L = D - Kappa

#eigvals, eigvecs=np.linalg.eigh(L)
#eigvecs = np.asarray(eigvecs)

#Ycurv = np.concatenate((eigvecs[:,1][:,None],eigvecs[:,2][:,None]),axis=1)
Ycurv=kernel_pca_.fit_transform(Kappa)

#ax = fig.add_subplot(132)
#ax.scatter(Y[:, 0], Y[:, 1], c=color, cmap=plt.cm.Spectral)
#plt.title("MDS alone")
#ax.xaxis.set_major_formatter(NullFormatter())
#ax.yaxis.set_major_formatter(NullFormatter())
#ax.axis('tight')

#ax = fig.add_subplot(133)
#ax.scatter(Ycurv[:, 0], Ycurv[:, 1], c=color, cmap=plt.cm.Spectral)
#plt.title("curvature + MDS")
#ax.xaxis.set_major_formatter(NullFormatter())
#ax.yaxis.set_major_formatter(NullFormatter())
#ax.axis('tight')

44850